In [7]:
import pandas as pd
import json
import os
from client import RestClient
import pprint

In [2]:
DFS_LOGIN = os.getenv('DFS_LOGIN')
print(DFS_LOGIN)
DFS_KEY = os.getenv('DFS_KEY')
print(DFS_KEY)

zl108108@gmail.com
742c0c2837362530


In [5]:
client = RestClient(DFS_LOGIN, DFS_KEY)
post_data = dict()
# simple way to set a task
post_data[len(post_data)] = dict(
    keywords=[
        "data analyst",
        "analytics engineer"
    ],
    location_name="Germany",
    language_name="German"
)
# POST /v3/dataforseo_labs/google/historical_search_volume/live
response = client.post("/v3/dataforseo_labs/google/historical_search_volume/live", post_data)
# you can find the full list of the response codes here https://docs.dataforseo.com/v3/appendix/errors
if response["status_code"] == 20000:
    print(response)
    # do something with result
else:
    print("error. Code: %d Message: %s" % (response["status_code"], response["status_message"]))

{'version': '0.1.20241203', 'status_code': 20000, 'status_message': 'Ok.', 'time': '0.1234 sec.', 'cost': 0.0102, 'tasks_count': 1, 'tasks_error': 0, 'tasks': [{'id': '12051148-5959-0401-0000-cfe20bb2764f', 'status_code': 20000, 'status_message': 'Ok.', 'time': '0.0816 sec.', 'cost': 0.0102, 'result_count': 1, 'path': ['v3', 'dataforseo_labs', 'google', 'historical_search_volume', 'live'], 'data': {'api': 'dataforseo_labs', 'function': 'historical_search_volume', 'se_type': 'google', 'keywords': ['data analyst', 'analytics engineer'], 'location_name': 'Germany', 'language_name': 'German'}, 'result': [{'se_type': 'google', 'location_code': 2276, 'language_code': 'de', 'items_count': 2, 'items': [{'se_type': 'google', 'keyword': 'analytics engineer', 'location_code': 2276, 'language_code': 'de', 'search_partners': False, 'keyword_info': {'se_type': 'google', 'last_updated_time': '2024-11-08 15:32:04 +00:00', 'competition': 0.27, 'competition_level': 'LOW', 'cpc': 3.99, 'search_volume': 2

In [6]:
response

{'version': '0.1.20241203',
 'status_code': 20000,
 'status_message': 'Ok.',
 'time': '0.1234 sec.',
 'cost': 0.0102,
 'tasks_count': 1,
 'tasks_error': 0,
 'tasks': [{'id': '12051148-5959-0401-0000-cfe20bb2764f',
   'status_code': 20000,
   'status_message': 'Ok.',
   'time': '0.0816 sec.',
   'cost': 0.0102,
   'result_count': 1,
   'path': ['v3',
    'dataforseo_labs',
    'google',
    'historical_search_volume',
    'live'],
   'data': {'api': 'dataforseo_labs',
    'function': 'historical_search_volume',
    'se_type': 'google',
    'keywords': ['data analyst', 'analytics engineer'],
    'location_name': 'Germany',
    'language_name': 'German'},
   'result': [{'se_type': 'google',
     'location_code': 2276,
     'language_code': 'de',
     'items_count': 2,
     'items': [{'se_type': 'google',
       'keyword': 'analytics engineer',
       'location_code': 2276,
       'language_code': 'de',
       'search_partners': False,
       'keyword_info': {'se_type': 'google',
        '

In [9]:
result = response.get('result')

In [11]:
print(result)


None


In [12]:
def extract_search_volume_to_json(json_data):
    """
    Extracts search volume data from the provided JSON object and formats it as a JSON structure.

    Args:
        json_data (dict): The JSON object containing search volume data.

    Returns:
        list: A list of dictionaries with extracted search volume data.
    """
    results = []
    for task in json_data.get("tasks", []):
        for result in task.get("result", []):
            for item in result.get("items", []):
                keyword = item.get("keyword")
                monthly_searches = item["keyword_info"].get("monthly_searches", [])
                for monthly in monthly_searches:
                    results.append({
                        "keyword": keyword,
                        "year": monthly["year"],
                        "month": monthly["month"],
                        "search_volume": monthly["search_volume"]
                    })
    return results

In [16]:
json_data = extract_search_volume_to_json(response)

In [18]:
json_data

[{'keyword': 'analytics engineer',
  'year': 2024,
  'month': 10,
  'search_volume': 210},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 9,
  'search_volume': 210},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 8,
  'search_volume': 170},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 7,
  'search_volume': 210},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 6,
  'search_volume': 170},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 5,
  'search_volume': 170},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 4,
  'search_volume': 210},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 3,
  'search_volume': 210},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 2,
  'search_volume': 260},
 {'keyword': 'analytics engineer',
  'year': 2024,
  'month': 1,
  'search_volume': 260},
 {'keyword': 'analytics engineer',
  'year': 2023,
  'month': 12,
  'search_volume': 210},
 {'keywo

In [20]:
def extract_search_volume(json_data):
    """
    Extracts search volume data from the provided JSON object.

    Args:
        json_data (dict): The JSON object containing search volume data.

    Returns:
        pd.DataFrame: A DataFrame with columns for Keyword, Year, Month, and Monthly Search Volume.
    """
    results = []
    for task in json_data.get("tasks", []):
        for result in task.get("result", []):
            for item in result.get("items", []):
                keyword = item.get("keyword")
                search_volume = item["keyword_info"].get("search_volume")
                monthly_searches = item["keyword_info"].get("monthly_searches", [])
                for monthly in monthly_searches:
                    results.append(
                        {
                            "Keyword": keyword,
                            "Year": monthly["year"],
                            "Month": monthly["month"],
                            "Monthly Search Volume": monthly["search_volume"],
                        }
                    )
                    
    return pd.DataFrame(results)

In [22]:
df = extract_search_volume(response)

In [23]:
df

,Keyword,Year,Month,Monthly Search Volume
0,analytics engineer,2024,10,210
1,analytics engineer,2024,9,210
2,analytics engineer,2024,8,170
3,analytics engineer,2024,7,210
4,analytics engineer,2024,6,170
...,...,...,...,...
141,data analyst,2019,2,5400
142,data analyst,2019,1,6600
143,data analyst,2018,12,4400
144,data analyst,2018,11,5400
